## Assignment 1, parts d) and e)
The text of the task can be found in the .pdf file with assignment description. The mathematical forumaltion of the problem will be presented here. Your task will be to implement this forumaltion in Python. At first, the whole forumlation will be given. Then, necessary parts will be shown at the respective parts of the code. 

---



Let's denote $z_0^p$: initial investment in project $p$ (1 variable per project) and $z_i^p$: continue project $p$ in scenario $i$ (2 variables per project).

If in implementation stage scenario i, $s_i, i \in {1,2}$ realizes, then in implemenation result the scenarios can be either $s_1$, $s_2$ or $s_3$.

Then, let's denote $ioS_0$ is the cash flow (remaining cash) after the first decision for each project, $ioS_{k}, k\in \{1,2,3\}$ is the total cash flow in scenario $k$ in implementaion stage immediately after making the decision to implement or to terminate for each project and $ioS_{ij}$ is the outcome in scenario $ij$ after the implementation result scenario outcome is known.

The optimisation problem can now be stated as
\begin{align*}
    \max_{z_i^p} & \sum_{i \in \{1,2\}, j \in \{1,2,3\}} P(s_{ij})iOS_{ij} \\
    \text{s.t.} & \\
    & ioS_0 = 9-2z_0^W - 3z_0^G \\
    & ioS_1 = 1.04\times ioS_0 - 2z_1^W - 2z_1^G \\
    & ioS_2 = 1.04\times ioS_0 - 2z_2^W - 2z_2^G \\
    & ioS_{11} = 1.04\times ioS_1 + 18z_1^W + 2z_1^G \\
    & ioS_{12} = 1.04\times ioS_1 + 14z_1^W + 3z_1^G \\
    & ioS_{13} = 1.04\times ioS_1 + 7z_1^W + 6z_1^G \\
    & ioS_{21} = 1.04\times ioS_2 + 4z_2^W + 19z_2^G \\
    & ioS_{22} = 1.04\times ioS_2 + 1.5z_2^W + 12z_2^G \\
    & z_0^W \geq z_1^W \\
    & z_0^W \geq z_2^W \\
    & z_0^G \geq z_1^G \\
    & z_0^G \geq z_2^G \\
    & z_i^p \in \{0,1\} \,  \forall i,p \\
    & ioS_i \geq 0 \, \forall i
\end{align*}



In [1]:
# Install gurobipy package. These cell must be executed at every launch of Google Colab. 
# DO NOT DELETE OR MODIFY THIS CELL
!pip install gurobipy

     ---------------------------------------- 8.9/8.9 MB 5.6 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\phamh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Import dependencies
# DO NOT DELETE OR MODIFY THIS CELL. 
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Initiate the model

model = gp.Model("assignmentA1.2")

In [ ]:
# Create binary variables representing the investment decisions, six in total

zW0 = model.addVar(vtype=GRB.BINARY, name="zW0") #initial investment decision for project Wind(W)
zG0 = model.addVar(vtype=GRB.BINARY, name="zG0") #initical investment decision for project Geothermal(G)
zW1 = model.addVar(vtype=GRB.BINARY, name="zW1") #continue project W in scenario 1 
zW2 = model.addVar(vtype=GRB.BINARY, name="zW2") #continue project W in scenario 2
zG1 = model.addVar(vtype=GRB.BINARY, name="zG1") #continue project G in scenario 1
zG2 = model.addVar(vtype=GRB.BINARY, name="zG2") #continue project G in scenario 2

In [ ]:
# Create real variables representing the investment outcomes, eight in total

iOS0 = model.addVar(vtype=GRB.CONTINUOUS, name="i0SO") #cash flow (remaining cash) after the first decision for each project
iOS1 = model.addVar(vtype=GRB.CONTINUOUS, name="i0S1") #total cash flow in scenario 1 in implementaion stage immediately after making the decision (continue or not)
iOS2 = model.addVar(vtype=GRB.CONTINUOUS, name="i0S2") #total cash flow in scenario 2 in implementaion stage immediately after making the decision (continue or not)
iOS11 = model.addVar(vtype=GRB.CONTINUOUS, name="i0S11") #the outcome in scenario 11 after the implementation result scenario outcome is known.
iOS12 = model.addVar(vtype=GRB.CONTINUOUS, name="i0S12") #the outcome in scenario 12 after the implementation result scenario outcome is known.
iOS13 = model.addVar(vtype=GRB.CONTINUOUS, name="i0S13") #the outcome in scenario 13 after the implementation result scenario outcome is known.
iOS21 = model.addVar(vtype=GRB.CONTINUOUS, name="i0S21") #the outcome in scenario 13 after the implementation result scenario outcome is known.
iOS22 = model.addVar(vtype=GRB.CONTINUOUS, name="i0S22") #the outcome in scenario 13 after the implementation result scenario outcome is known.

In [ ]:
# Define probabilities in the scenario tree
P1 = 0.5
P2 = 0.5

P11 = P1*0.4
P12 = P1*0.2
P13 = P1*0.4

P21 = P2*0.3
P22 = P2*0.7

In [ ]:
# Define interest
r = 1.04

In [ ]:
# Define cash flows for different outcomes
cfWs11 = 18
cfWs12 = 14
cfWs13 = 7

cfWs21 = 4
cfWs22 = 1.5

cfGs11 = 2
cfGs12 = 3
cfGs13 = 6

cfGs21 = 19
cfGs22 = 12

In [ ]:
# Define initial investments at evaluation stage
#iEW = 2
iEW = 1.5 #for task E
#iEG = 3
iEG = 2 #for task E
# Define initial investments at imlementation stage
#iIW = 2
iIW = 3.5 #for task E
iIG = 2

# Define allocated for investments
totalInv = 9

Let's set the objective function
\begin{align}
    \max_{z_i^p} & \sum_{i \in \{1,2\}, j \in \{1,2,3\}} P(s_{ij})iOS_{ij} \\
\end{align}

In [ ]:
# Set an objective function, don't forget to mention the type of problem (minimization/maximization)
model.setObjective(P11*iOS11 + P12*iOS12 + P13*iOS13 + P21*iOS21 + P22*iOS22, GRB.MAXIMIZE)

Let's add the constrain for the outcome in evaluation stage
\begin{align*}
    & ioS_0 = 9-2z_0^W - 3z_0^G \\
\end{align*}

In [ ]:
# Calculate the outcome in the evaluation stage and add it as a constrain
model.addConstr(iOS0 == totalInv-iEW*zW0-iEG*zG0)

<gurobi.Constr *Awaiting Model Update*>

Let's add the constrain for the outcomes in implementation stage
\begin{align*}
    & ioS_1 = 1.04\times ioS_0 - 2z_1^W - 2z_1^G \\
    & ioS_2 = 1.04\times ioS_0 - 2z_2^W - 2z_2^G \\
    & ioS_i \geq 0 \, \forall i
\end{align*}

In [ ]:
# Calculate outcomes in the implementation stage and add them as constrains

model.addConstr(iOS1 == r*iOS0 - iIW*zW1 - iIG*zG1)
model.addConstr(iOS2 == r*iOS0 - iIW*zW2 - iIG*zG2)
#model.addConstr(iOS)

<gurobi.Constr *Awaiting Model Update*>

Let's add the constrain for the outcomes after implementation stage
\begin{align*}
    & ioS_{11} = 1.04\times ioS_1 + 18z_1^W + 2z_1^G \\
    & ioS_{12} = 1.04\times ioS_1 + 14z_1^W + 3z_1^G \\
    & ioS_{13} = 1.04\times ioS_1 + 7z_1^W + 6z_1^G \\
    & ioS_{21} = 1.04\times ioS_2 + 4z_2^W + 19z_2^G \\
    & ioS_{22} = 1.04\times ioS_2 + 1.5z_2^W + 12z_2^G \\
\end{align*}

In [ ]:
# Calculate outcomes after the implementation stage using cash flows for different scenarios and add them as constrains

# Constraints for the scenario s1 
model.addConstr(iOS11 == r*iOS1 + cfWs11*zW1 + cfGs11*zG1)
model.addConstr(iOS12 == r*iOS1 + cfWs12*zW1 + cfGs12*zG1)
model.addConstr(iOS13 == r*iOS1 + cfWs13*zW1 + cfGs13*zG1)
# Constraints doe the scenario s2
model.addConstr(iOS21 == r*iOS2 + cfWs21*zW2 + cfGs21*zG2)
model.addConstr(iOS22 == r*iOS2 + cfWs22*zW2 + cfGs22*zG2)

<gurobi.Constr *Awaiting Model Update*>

Let's add consistency constraints for decisions
\begin{align*}
    & z_0^W \geq z_1^W \\
    & z_0^W \geq z_2^W \\
    & z_0^G \geq z_1^G \\
    & z_0^G \geq z_2^G \\
\end{align*}

In [ ]:
# Add consistency constraints for decisions
# model.addConstr(...)

model.addConstr(zW0 >= zW1)
model.addConstr(zW0 >= zW2)
model.addConstr(zG0 >= zG1)
model.addConstr(zG0 >= zG2)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Optimize the model

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 12 rows, 14 columns and 39 nonzeros
Model fingerprint: 0x3f325fdb
Variable types: 8 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-01, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 9e+00]
Presolve removed 12 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 17.3988 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.739880000000e+01, best bound 1.739880000000e+01, gap 0.0000%


In [ ]:
# Print out solutions, first 6 are decisions, next is evaluation outcome, next 2 are implementation outcomes and last 5 are for implementation result outcomes

for v in model.getVars():
  print('%s = %g' % (v.varName, v.x))


zW0 = 1
zG0 = 1
zW1 = 1
zW2 = 0
zG1 = 1
zG2 = 1
i0SO = 5.5
i0S1 = 0.22
i0S2 = 3.72
i0S11 = 20.2288
i0S12 = 17.2288
i0S13 = 13.2288
i0S21 = 22.8688
i0S22 = 15.8688


In [ ]:
# Print out the expected cash flow from the project (the value of objective function). We do not take into account the time value of money. 

print('Obj: %g' % model.objVal)

Obj: 17.3988
